In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image

In [ ]:
import tensorflow as tf

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D

In [4]:
file_path = os.listdir('/Users/adithyaram/Downloads/dataset')
print(len(file_path))

4738


In [5]:
train_data, test_data = train_test_split(file_path, test_size = 0.15)

In [6]:
print("Number of Training Images:",len(train_data))
print("Number of Testing Images:",len(test_data))

Number of Training Images: 4027
Number of Testing Images: 711


In [7]:
def imagetoarray(file_array):
    images_array = np.empty((len(file_array), 224, 224, 3), dtype=np.float32)
    
    for i,path in enumerate(file_array):
        image_path = ('/Users/adithyaram/Downloads/dataset/' + path)
        image = Image.open(image_path)
        resized_image = image.resize((224, 224))
        resized_image = np.array(resized_image)
        images_array[i] = resized_image.astype('float32')/255
        
    return images_array
        

In [8]:
train_data = imagetoarray(train_data)
print("Length of training dataset:",train_data.shape)
test_data = imagetoarray(test_data)
print("Length of test dataset:",test_data.shape)

Length of training dataset: (4027, 224, 224, 3)
Length of test dataset: (711, 224, 224, 3)


In [9]:
def encoder_decoder_model():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3), padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    model.add(Conv2D(128, kernel_size=(3, 3), kernel_regularizer = tf.keras.regularizers.L2(0.001), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')) 
    model.add(Conv2D(256, kernel_size=(3, 3), kernel_regularizer = tf.keras.regularizers.L2(0.001), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')) 
    model.add(Conv2D(512, kernel_size=(3, 3), kernel_regularizer = tf.keras.regularizers.L2(0.001), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid')) 
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid')) 
    
    model.add(Conv2D(512, kernel_size=(3, 3), kernel_regularizer = tf.keras.regularizers.L2(0.001), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))        
    model.add(Conv2D(512, kernel_size=(3, 3), kernel_regularizer = tf.keras.regularizers.L2(0.001), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(256, kernel_size=(3, 3), kernel_regularizer = tf.keras.regularizers.L2(0.001), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))              
    model.add(Conv2D(128, kernel_size=(3, 3), kernel_regularizer = tf.keras.regularizers.L2(0.001), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))            
    model.add(Conv2D(64, kernel_size=(3, 3), kernel_regularizer = tf.keras.regularizers.L2(0.001), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))  
    model.add(Conv2D(3, kernel_size=(3, 3), kernel_regularizer = tf.keras.regularizers.L2(0.001), activation='relu', padding='same'))
    return model    

In [10]:
model = encoder_decoder_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 28, 28, 256)       0

In [13]:
from tensorflow.keras.optimizers.legacy import Adam

In [14]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

In [ ]:
model.fit(train_data, train_data, epochs=10, batch_size=32)

Epoch 1/10
126/126 [==============================] - 928s 7s/step - loss: 0.5964
Epoch 2/10
126/126 [==============================] - 47305s 378s/step - loss: 0.2103
Epoch 3/10
126/126 [==============================] - 7052s 48s/step - loss: 0.1252
Epoch 4/10
126/126 [==============================] - 841s 7s/step - loss: 0.0841
Epoch 5/10
 90/126 [====================>.........] - ETA: 1:13:44 - loss: 0.0641

In [ ]:
from keras import backend as K

def feature_extraction(model, data, layer):

    encoded = K.function([model.layers[0].input],[model.layers[layer].output])
    encoded_array = encoded([data])[0]
    pooled_array = encod= ed_array.max(axis=-1)
    return encoded_array

encoded = feature_extraction(model,train_data[:10],9)

In [ ]:
from sklearn.manifold import TSNE


tsne = TSNE(n_components=2)
values = tsne.fit_transform(encoded) 


In [ ]:
from sklearn.cluster import KMeans

In [1]:
K = [4, 5, 6, 7]

for k in K:
    kmeans = KMeans(n_clusters = k, random_state=0).fit(encoded)
    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_
    
    plt.figure(figsize=(10,5)) 
    plt.subplot(1,1,1)
    plt.scatter(values[:,0], values[:,1], c= kmeans.labels_.astype(float), s=50, alpha=0.5)
    plt.scatter(centroids[:, 0], centroids[:, 1], c=None, s=50)
    plt.show()
    for row in range(k): 
        iter=0
        plt.figure(figsize=(13,3))
        for i,iterator in enumerate(labels):
            if iterator == row:
                img_path = '/Users/adithyaram/Downloads/dataset/' + lips[i]
                img = Image.open(img_path)
                plot_(img,"","",1,6,iter+1,"cluster="+str(row),"","","",True)
                iter+=1
            if iter>=5: break
        plt.show()
    print() 

IndentationError: expected an indented block (3674646434.py, line 3)